In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import __version__ as sklearn_version
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.dummy import DummyRegressor
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

In [2]:
df = pd.read_csv('../data/interim/salary_skill_cleaned.csv')
df.head()

,Unnamed: 0,occupation,skill,importance,location,state,salary
0,0,Chief Executives,Reading Comprehension,4.12,Aguadilla-Isabela,PR,92200.0
1,1,Chief Executives,Reading Comprehension,4.12,Akron,OH,204420.0
2,2,Chief Executives,Reading Comprehension,4.12,Albany,GA,121260.0
3,3,Chief Executives,Reading Comprehension,4.12,Albany-Schenectady-Troy,NY,155870.0
4,4,Chief Executives,Reading Comprehension,4.12,Allentown-Bethlehem-Easton,PA,215910.0


In [3]:
skills = df.pivot_table(values='importance',index=['occupation', 'state', 'salary'],columns='skill').reset_index()
skills.head()

skill,occupation,state,salary,Active Learning,Active Listening,Complex Problem Solving,Coordination,Critical Thinking,Equipment Maintenance,Equipment Selection,...,Science,Service Orientation,Social Perceptiveness,Speaking,Systems Analysis,Systems Evaluation,Technology Design,Time Management,Troubleshooting,Writing
0,Accountants and Auditors,AK,71610.0,3.31,3.94,3.44,3.06,3.88,1.0,1.0,...,1.88,3.0,3.06,3.81,3.19,2.94,1.75,3.25,1.12,3.75
1,Accountants and Auditors,AK,81170.0,3.31,3.94,3.44,3.06,3.88,1.0,1.0,...,1.88,3.0,3.06,3.81,3.19,2.94,1.75,3.25,1.12,3.75
2,Accountants and Auditors,AL,58570.0,3.31,3.94,3.44,3.06,3.88,1.0,1.0,...,1.88,3.0,3.06,3.81,3.19,2.94,1.75,3.25,1.12,3.75
3,Accountants and Auditors,AL,60400.0,3.31,3.94,3.44,3.06,3.88,1.0,1.0,...,1.88,3.0,3.06,3.81,3.19,2.94,1.75,3.25,1.12,3.75
4,Accountants and Auditors,AL,62120.0,3.31,3.94,3.44,3.06,3.88,1.0,1.0,...,1.88,3.0,3.06,3.81,3.19,2.94,1.75,3.25,1.12,3.75


In [4]:
skills.shape

(136041, 38)

In [5]:
# Drop Installation, Equipment Selection, Equipment Maintenance, and Repairing (from EDA)
skills.drop(columns=['Installation', 'Equipment Selection', 'Equipment Maintenance', 'Repairing'], inplace=True)
skills.set_index('occupation', inplace=True)
skills.columns

Index(['state', 'salary', 'Active Learning', 'Active Listening',
       'Complex Problem Solving', 'Coordination', 'Critical Thinking',
       'Instructing', 'Judgment and Decision Making', 'Learning Strategies',
       'Management of Financial Resources', 'Management of Material Resources',
       'Management of Personnel Resources', 'Mathematics', 'Monitoring',
       'Negotiation', 'Operation and Control', 'Operations Analysis',
       'Operations Monitoring', 'Persuasion', 'Programming',
       'Quality Control Analysis', 'Reading Comprehension', 'Science',
       'Service Orientation', 'Social Perceptiveness', 'Speaking',
       'Systems Analysis', 'Systems Evaluation', 'Technology Design',
       'Time Management', 'Troubleshooting', 'Writing'],
      dtype='object', name='skill')

In [6]:
skills.shape

(136041, 33)

In [7]:
# One-hot encode states
skills_coded = pd.get_dummies(skills, 
               columns=['state'], 
               drop_first=True, 
               prefix='C')
skills_coded.head()

,salary,Active Learning,Active Listening,Complex Problem Solving,Coordination,Critical Thinking,Instructing,Judgment and Decision Making,Learning Strategies,Management of Financial Resources,...,C_SD,C_TN,C_TX,C_UT,C_VA,C_VT,C_WA,C_WI,C_WV,C_WY
occupation,,,,,,,,,,,,,,,,,,,,,
Accountants and Auditors,71610.0,3.31,3.94,3.44,3.06,3.88,2.88,3.5,2.94,2.56,...,0,0,0,0,0,0,0,0,0,0
Accountants and Auditors,81170.0,3.31,3.94,3.44,3.06,3.88,2.88,3.5,2.94,2.56,...,0,0,0,0,0,0,0,0,0,0
Accountants and Auditors,58570.0,3.31,3.94,3.44,3.06,3.88,2.88,3.5,2.94,2.56,...,0,0,0,0,0,0,0,0,0,0
Accountants and Auditors,60400.0,3.31,3.94,3.44,3.06,3.88,2.88,3.5,2.94,2.56,...,0,0,0,0,0,0,0,0,0,0
Accountants and Auditors,62120.0,3.31,3.94,3.44,3.06,3.88,2.88,3.5,2.94,2.56,...,0,0,0,0,0,0,0,0,0,0


In [8]:
# Define X and y
X = skills_coded.drop(columns=['salary'])
y = skills_coded['salary']

# Train/Test Split

In [9]:
len(X) * .7, len(X) * .3

(95228.7, 40812.299999999996)

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=47)

In [11]:
X_train.shape, X_test.shape

((95228, 82), (40813, 82))

In [12]:
y_train.shape, y_test.shape

((95228,), (40813,))

In [13]:
X_train.dtypes

Active Learning            float64
Active Listening           float64
Complex Problem Solving    float64
Coordination               float64
Critical Thinking          float64
                            ...   
C_VT                         uint8
C_WA                         uint8
C_WI                         uint8
C_WV                         uint8
C_WY                         uint8
Length: 82, dtype: object

In [14]:
X_test.dtypes

Active Learning            float64
Active Listening           float64
Complex Problem Solving    float64
Coordination               float64
Critical Thinking          float64
                            ...   
C_VT                         uint8
C_WA                         uint8
C_WI                         uint8
C_WV                         uint8
C_WY                         uint8
Length: 82, dtype: object

In [15]:
skills_coded.to_csv('../data/processed/salary_skill_processed.csv')

In [16]:
X_train.to_csv('../data/processed/X_train.csv')
y_train.to_csv('../data/processed/y_train.csv')
X_test.to_csv('../data/processed/X_test.csv')
y_test.to_csv('../data/processed/y_test.csv')